# Latent Dirichlet Allocation (LDA) Pipeline Example in Python

Below is a tutorial on how to process data and to train an LDA on it. 

Knowing that the LDA perform better with stop words removal and with lemmatized words, it's results are often ugly on the lemmatized words. To fix that, it's a good thing to be able to do an inverse-lemmatization on the topic words (or topic n-grams) yield by the LDA once trained on the data. 

So here we are: let's do a pipeline that looks like that: 

1. Load a dataset of many comments (or documents)
2. Transform comments to remove stop words
3. Lemmatize the comments without stop words for a better LDA
4. Perform LDA topic modeling
5. Recover words from inverse (backwards) lemmatization on topic words 
6. Clean topic words are available

Note: The classes imported are clean and have unit tests. Don't hesitate to dive in and to check what's under the hood!


In [ ]:

from app.data.load_sample_data import load_sample_data
from app.logic.stop_words_remover import StopWordsRemover
from app.logic.stemmer import Stemmer
from app.logic.lda import LDA
from app.logic.count_vectorizer import CountVectorizer


## Load a dataset of many comments (or documents)

In [ ]:
ids, messages, comments = load_sample_data()

## Transform comments to remove stop words

In [ ]:
fr_en_stopwords = StopWordsRemover()
comments_without_stopwords = fr_en_stopwords.transform(comments)

## Lemmatize the comments without stop words for clean texts

In [ ]:
french_stemmer = Stemmer(language='french')
stemmed_comments = [french_stemmer.fit_transform(thread) for thread in comments_without_stopwords]

## Perform LDA topic modeling

In [ ]:
# lda = LDA(n_components=2, max_iter=5, learning_method='online', learning_offset=50.)
# lda_sklearn, feature_names = lda.fit(stemmed_comments[-4])

## Recover words from inverse (backwards) lemmatization on topic words 

## Some clean topic words (or expressions) are then available!

## TODO: clean below

In [ ]:
i = -2
chosen_id = ids[i]
chosen_message = messages[i]
chosen_comments = comments[i]
# chosen_id, chosen_message, chosen_comments, str(len(chosen_comments)) + " comments."

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


lda_pipeline = Pipeline([
    ('stopwords', StopWordsRemover()),
    ('stemmer', Stemmer()),
    # ('letter_splitter', LetterSplitter()),
    ('count_vect', CountVectorizer()),
    ('lda', LDA()),
])

# Params where many options are in lists represents choice in the later grid search. 
lda_pipeline_params = {
    'stemmer__language': ['french'],
    'count_vect__max_df': [0.98, 0.95],
    'count_vect__min_df': [2, 3],
    'count_vect__max_features': [10000],
    'count_vect__ngram_range': [(1, 1), (1, 2)],
    'count_vect__strip_accents': ['ascii', 'unicode', None],
    'lda__n_components': [2],
    'lda__max_iter': [100],  # TODO: find good balance here.
    'lda__learning_decay': [0.5, 0.7, 1.0],
    'lda__learning_method': ['online'],
    'lda__learning_offset': [5, 10],
    'lda__batch_size': [1, 10, 25],
    # 'lda__n_jobs': -1,  # Use all CPUs
}
# """
lda_pipeline_params = {
    'stemmer__language': ['french'],
    'count_vect__max_df': [0.98],
    'count_vect__min_df': [2],
    'count_vect__max_features': [10000],
    'count_vect__ngram_range': [(1, 2)],
    'count_vect__strip_accents': [None],
    'lda__n_components': [2],
    'lda__max_iter': [1000],  # TODO: find good balance here.
    'lda__learning_decay': [0.5],
    'lda__learning_method': ['online'],
    'lda__learning_offset': [10],
    'lda__batch_size': [25],
    # 'lda__n_jobs': -1,  # Use all CPUs
}
# """


# TODO: Might need to code a custom parameter search
gs = GridSearchCV(lda_pipeline, lda_pipeline_params, n_jobs=-1, verbose=1, cv=2)
gs.fit(chosen_comments)
print("Best score: {}".format(gs.best_score_))



best_params = gs.best_estimator_.get_params()
# import pprint
# pp = pprint.PrettyPrinter(indent=4)
# print(type(best_params))
# pp.pprint(best_params)

best_pipeline = Pipeline(best_params['steps'])
transformed_comments = best_pipeline.fit_transform(chosen_comments)
print("score:", best_pipeline.score(chosen_comments))

features = best_pipeline.named_steps['count_vect'].get_feature_names()
lda = best_pipeline.named_steps['lda']



In [ ]:
lda = best_pipeline.named_steps['lda']
topics = lda.components_
# print(topics)
print("")
topic_words = best_pipeline.inverse_transform(X=None)
print("")
print(topic_words[0])
print(topic_words[1])

In [ ]:
# Some print-info to clean later on

from app.application.data_utils import link_topics_and_weightings

print("# Unique ID:")
print(chosen_id)
print("\n# Message:")
print(chosen_message)
print("\n# Comments:")
print(chosen_comments)
print("\n# Topics' ponderation in each comment:")
print(transformed_comments)
print("\n# Topics' words:")
print(topic_words)
print("\n# Topic words' weighting:")
topic_words_weighting = [list(reversed(sorted(t))) for t in topics]
print(topic_words_weighting)
print("\n# Alignment of topic words with their weightings (same as previous two arrays)")
print(link_topics_and_weightings(topic_words, topic_words_weighting))
print("\nNote: you should show only the first half of the topics' words or less, because the word list for topic includes EVERY word, so each topic have all the words but with different weightings. "
      "Normally I'd trim that to a maximum word count or weighting threshold not to return every words.")


In [ ]:
import numpy as np
print(np.array(chosen_comments))
print("\nSplitting on topics as if they were booleans:")
print(transformed_comments.argmax(1))

group_1 = np.array(chosen_comments)[0==transformed_comments.argmax(1).astype(int)]
group_2 = np.array(chosen_comments)[1==transformed_comments.argmax(1).astype(int)]

print("\nComments splitted:")
print(group_1)
print(group_2)

In [ ]:
# # messages[-2], comments[-2]
# # features = best_pipeline.named_steps['count_vect'].get_feature_names()

In [ ]:

lda = best_pipeline.named_steps['lda']
count_vect = best_pipeline.named_steps['count_vect']
stemmer = best_pipeline.named_steps['stemmer']

lda.print_top_words(count_vect.get_feature_names())
print("")

print(lda.components_)
topics = lda.inverse_transform()
print(topics)
topics_words = count_vect.inverse_transform(topics)
print(topics_words)
final_words = stemmer.inverse_transform(topics_words)
print(final_words[0])
print(final_words[1])


In [ ]:

# train, test = sklearn.model_selection.train_test_split(stemmed_comments[-2], test_size=0.2)
# print(len(train), len(test))

# lda_sklearn, feature_names = lda.fit(train)

# print(lda.score(train), lda.perplexity(train))
# print(lda.score(test), lda.perplexity(test))
# -182.51029019702543   27.615270885701467
# -52.05494670051718    41.19061662552155   
# -41.973876576664225  189.9450029464927    
# -32.432059761463556 3320.9791408284505    
# perplexity should go down and score should go down too (more negative). 

In [ ]:
# len(stemmed_comments[-2]), len(stemmed_comments[-3]), len(stemmed_comments[-4])

In [ ]:
# ([(len(i), a) for a, i in enumerate(stemmed_comments)])

In [ ]:
# fr_en_stopwords = StopWordsRemover()
# fr_en_stopwords.remove_from_string("Le chat s'est assis sur le tapis aujour'hui!!! il est très comfortable et n'est pas déçu, tout ronron!")